In [24]:
import sklearn as sk
import pandas as pd
import numpy as np
import statistics
import math
import os
from sklearn import preprocessing
from sklearn.utils.random import sample_without_replacement
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import IntProgress
from IPython.display import display
import time
import pickle
from scipy.special import logit
import shap
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm


import warnings
from shap.plots._labels import labels
from shap.utils import format_value, ordinal_str
from shap.plots._utils import convert_ordering, convert_color, merge_nodes, get_sort_order, sort_inds, dendrogram_coords
from shap.plots import colors
import scipy
import copy
from shap._explanation import Explanation, Cohorts

In [25]:
%matplotlib notebook

In [26]:
# Complex model
def experiment_data_complex(p,n): 
    
    A = np.random.binomial(1,p,n) #gender roles
    B = [np.random.binomial(1,0.5,1)[0] if x ==0 else np.random.binomial(1,0.7,1)[0] for x in A] 
    C = [np.random.binomial(1,0.01,1)[0] if x ==0 else np.random.binomial(1,0.99,1)[0] for x in B] 
    D = [np.random.binomial(1,0.8,1)[0] if x ==0 else np.random.binomial(1,0.4,1)[0] for x in A] 
    E = [np.random.binomial(1,0.09,1)[0] if x ==0 else np.random.binomial(1,0.82,1)[0] for x in C]

    df = pd.DataFrame({'A': A, 'B': B, 'C': C, 'D': D, 'E': E})
    
    P = 0.15*df['A'] +  0.25*df['B'] +  0.1*df['C'] + 0.2*df['D'] +  0.30*df['E']
    
    Y = [np.random.binomial(1,p,1)[0] for p in P] 
    df['Y'] = Y
    
   
    
    return df

# Simple model
def experiment_data_simple(p,n): 
    
    A = np.random.binomial(1,p,n) #gender roles
    B = [np.random.binomial(1,0.5,1)[0] if x ==0 else np.random.binomial(1,0.7,1)[0] for x in A] 
    C = [np.random.binomial(1,0.01,1)[0] if x ==0 else np.random.binomial(1,0.99,1)[0] for x in B] 
#     D = [np.random.binomial(1,0.8,1)[0] if x ==0 else np.random.binomial(1,0.4,1)[0] for x in A] 
#     E = [np.random.binomial(1,0.09,1)[0] if x ==0 else np.random.binomial(1,0.82,1)[0] for x in C]

    df = pd.DataFrame({'A': A, 'B': B, 'C': C})
    
    P = 0.5*df['A'] +  0.2*df['B'] +  0.3*df['C']# + 0.2*df['D'] +  0.30*df['E']
    
    Y = [np.random.binomial(1,p,1)[0] for p in P] 
    df['Y'] = Y
    
   
    
    return df


In [27]:
def class_data(p,w1,w2, n): # high w1 good for group 1, high w2 good for group 0, have to add up to 0.8
    
    df = {}
    
   
    
    A = np.random.binomial(1,p,n) #sensitive attribute
    B1 = [np.random.binomial(1,0.2,1)[0] if x ==0 else np.random.binomial(1,0.9,1)[0] for x in A] #dependent on A
    B2 = [np.random.binomial(1,0.2,1)[0] if x ==1 else np.random.binomial(1,0.9,1)[0] for x in A] #dependent on A
    
    C1 = [*np.random.binomial(1,0.5,n)]#not dependent on A
    C2 = [*np.random.binomial(1,0.4,n)]#not dependent on A

    df = pd.DataFrame({'A': A,'B1':B1,'B2':B2, 'C1': C1, 'C2': C2})

    P = w1*df['B1']+ w2*df['B2'] + 0.10*df['C1']+0.10*df['C2']
 

    Y = [*np.random.binomial(1,P,n)]
#     Y=[]
#     for p in P:
#         if p>1:
#             p=1
#         y = np.random.binomial(1,p,1)
#         Y.append(y[0])
    df['Y'] = Y
        
   
    
    return df


In [34]:
def getPopSD():
    P0 = df_pop.loc[(df_pop[sensitive]==0)&(df_pop[label] ==1)].shape[0]/df_pop.loc[df_pop[sensitive]==0].shape[0]
    P1 =df_pop.loc[(df_pop[sensitive]==1)&(df_pop[label] ==1)].shape[0]/df_pop.loc[df_pop[sensitive]==1].shape[0]
    return P1 - P0
def getPopRatio():
    m0 = df_pop.loc[df_pop[sensitive]==0].shape[0]/df_pop.shape[0]
    m1 = df_pop.loc[df_pop[sensitive]==1].shape[0]/df_pop.shape[0]
    return m0,m1

In [35]:

# Functions to generate samples while varying the size. 
# Used mainly for SSB experiments

def sampling_sizes(sizes, df):
    '''
    Number of samples depend on sample size
    '''

    all_samples = {}
 
    for size in sizes:
        #print(size)
        df = df.sample(frac = 1) #shuffle only for each size ONCE
        s_num = df.shape[0]//size #number of samples depending on the sample size
        start = [x for x in range(0,s_num*size,size)]
        end = [x for x in range(size,s_num*size,size)]
        samples=[]
        for s,e in zip(start,end):
            df = df.sample(frac = 1)
            #print(s,e)
            s=df.iloc[s:e]
            samples.append(s)
        all_samples[size] = samples
    return all_samples

def validSample(s):
    return (s[s[label] == 0].shape[0] > 0) and (s[s[label] == 1].shape[0] > 0)

def sampling_overlaping(seeds, sizes, df):
    '''
    Same number of samples for all sample sizes
    '''
    all_samples = {}
    for size in sizes:
        #print('size = ' + str(size))
        samples = []
        for seed in seeds: 
#             print('df.size: ' + str(df.size))
#             print('sampling size: ' + str(size))
            goodSample = False
            attempts = 0
            while ((goodSample == False) and (attempts < 100)):
                s = df.sample(size, replace=True)
                if (validSample(s)):
                    goodSample = True
                attempts += 1
            #s = df.sample(size, replace=False, random_state=seed)
            samples.append(s)
        all_samples[size]=samples
        
    return all_samples

In [36]:
# Function to retrieve the feature importance value
def get_importance(sensitive,shap_values, max_display=10, order=Explanation.abs, clustering=None):
    if isinstance(shap_values, Explanation):
        cohorts = {"": shap_values}
    elif isinstance(shap_values, Cohorts):
        cohorts = shap_values.cohorts
    else:
        assert isinstance(shap_values,
                          dict), "You must pass an Explanation object, Cohorts object, or dictionary to bar plot!"

    # unpack our list of Explanation objects we need to plot
    cohort_labels = list(cohorts.keys())
    cohort_exps = list(cohorts.values())
    for i in range(len(cohort_exps)):
        if len(cohort_exps[i].shape) == 2:
            cohort_exps[i] = cohort_exps[i].abs.mean(0)
        assert isinstance(cohort_exps[i],
                          Explanation), "The shap_values paramemter must be a Explanation object, Cohorts object, or dictionary of Explanation objects!"
        assert cohort_exps[i].shape == cohort_exps[
            0].shape, "When passing several Explanation objects they must all have the same shape!"
        # TODO: check other attributes for equality? like feature names perhaps? probably clustering as well.
        
        
    # unpack the Explanation object
    features = cohort_exps[0].data
    feature_names = cohort_exps[0].feature_names
    if clustering is None:
        partition_tree = getattr(cohort_exps[0], "clustering", None)
    elif clustering is False:
        partition_tree = None
    else:
        partition_tree = clustering
    if partition_tree is not None:
        assert partition_tree.shape[
                   1] == 4, "The clustering provided by the Explanation object does not seem to be a partition tree (which is all shap.plots.bar supports)!"
    op_history = cohort_exps[0].op_history
    values = np.array([cohort_exps[i].values for i in range(len(cohort_exps))])

    if len(values[0]) == 0:
        raise Exception("The passed Explanation is empty! (so there is nothing to plot)")

  

    # TODO: Rather than just show the "1st token", "2nd token", etc. it would be better to show the "Instance 0's 1st but", etc
    if issubclass(type(feature_names), str):
        feature_names = [ordinal_str(i) + " " + feature_names for i in range(len(values[0]))]
        
        
    # find how many instances are in each cohort (if they were created from an Explanation object)
    cohort_sizes = []
    for exp in cohort_exps:
        for op in exp.op_history:
            if op.get("collapsed_instances", False):  # see if this if the first op to collapse the instances
                cohort_sizes.append(op["prev_shape"][0])
                break

    # unwrap any pandas series
    if str(type(features)) == "<class 'pandas.core.series.Series'>":
        if feature_names is None:
            feature_names = list(features.index)
        features = features.values

    # ensure we at least have default feature names
    if feature_names is None:
        feature_names = np.array([labels['FEATURE'] % str(i) for i in range(len(values[0]))])
        
    orig_inds = [[i] for i in range(len(values[0]))]
    orig_values = values.copy()

    feature_order = np.argsort(np.mean([np.argsort(convert_ordering(order, Explanation(values[i]))) for i in range(values.shape[0])], 0))

    

    
    
    
    # here we build our feature names, accounting for the fact that some features might be merged together
    feature_inds = feature_order[:max_display]
    y_pos = np.arange(len(feature_inds), 0, -1)
    feature_names_new = []
    for pos, inds in enumerate(orig_inds):
        if len(inds) == 1:
            feature_names_new.append(feature_names[inds[0]])
        else:
            full_print = " + ".join([feature_names[i] for i in inds])
            if len(full_print) <= 40:
                feature_names_new.append(full_print)
            else:
                max_ind = np.argmax(np.abs(orig_values).mean(0)[inds])
                feature_names_new.append(feature_names[inds[max_ind]] + " + %d other features" % (len(inds) - 1))
    feature_names = feature_names_new


    sens_ind = feature_names.index(sensitive)
    #print(sens_ind)
    for i in range(len(values)):

        ind = feature_order[i]
        imp = round(values[i, sens_ind],2)
    return imp

In [37]:
# Utility functions used in the main classification function (classifier_biases)
def computeVariance(preds):
    nbrSamples = len(preds)
    sizeX = len(preds[0])
    
    return math.sqrt(statistics.mean([statistics.variance([float(x) for x in preds[:][i]]) for i in range(nbrSamples)]))

def computeZOL(df, group):
    
    df_t = df.loc[(df[sensitive] == group)]
    
    counter = 0
    for i in range(df_t.shape[0]):
        currentEntry = df_t.iloc[i]
        if (currentEntry['predictions'] != currentEntry[label]):
            counter+=1
    
    return counter/df_t.shape[0]
    

def cm_metrics(y_true, y_pred, epsilon=0.0001):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    tpr = tp/(tp+fn) #true positive rate
    fpr = fp/(fp+tn)#false positive rate
    fnr = fn/(tp+fn)#false negative rate
    tnr = tn/(tn+fp)
    precision = tp/(tp+fp)
    
    return tpr, fpr,fnr, tnr, precision      
    


In [64]:
def classifier_biases(samples, X_test,Y_test, classifier, sensitiveImportance=False):
    #classifier:
    # rf: RandomForestClassifier()
    # lg: LogisticRegression()
    # lr: LinearRegression()
    # etc.
 
    m=0
    
    FPR0 = {}
    FPR1 = {}
    
    FNR0 = {}
    FNR1 = {}
    
    EO0 = {}
    EO1 = {}
    
    SD0 = {}
    SD1 = {}
     
    ZOL0 = {}
    ZOL1 = {}
    
    VarFPR0 = {}
    VarFPR1 = {}
    
    VarFNR0 = {}
    VarFNR1 = {}
    
    VarEO0 = {}
    VarEO1 = {}
    
    VarSD0 = {}
    VarSD1 = {}
    
    VarZOL0 = {}
    VarZOL1 = {}
    
    VarDisc_FPR = {}
    VarDisc_EO = {}
    VarDisc_ZOL = {}
    VarDisc_AUC = {}
    VarDisc_SD = {}
    
    fpr_roc_data0 = {}
    fpr_roc_data1 = {}
    
    tpr_roc_data0 = {}
    tpr_roc_data1 = {}
    
    auc0 = {}
    auc1 = {} 
    
    Var_auc0 = {}
    Var_auc1 = {}
    
    shapVal = {}
    varShapVal = {}
       
    max_count = len(samples)
    f = IntProgress(min=0, max=max_count) # instantiate the bar
    display(f) # display the bar
    
    for key,value in samples.items(): # loop over sizes
        f.value += 1
        
        fpr0_temp = []
        fpr1_temp = []
        #fpr_temp = []
        
        fnr0_temp = []
        fnr1_temp = []
        #fnr_temp = []
        
        eo0_temp = []
        eo1_temp = []
        
        sd0_temp = []
        sd1_temp = []
        #sd_temp = []
        
        zol0_temp = []
        zol1_temp = []
        
        fpr0_roc_temp = []
        fpr1_roc_temp = []
        tpr0_roc_temp = []
        tpr1_roc_temp = []
        
        auc0_temp = []
        auc1_temp = []  
              
        shapVal_temp = []
        
        Disc_FPR_temp = []
        Disc_ZOL_temp = []
        Disc_EO_temp = []
        Disc_AUC_temp = []
        Disc_SD_temp = []
       
        
        
        for v in value: # loop over samples of the same size
            data=v
            m = len(data)
            X = data.drop(label, axis=1)
            Y = data[label]
            
            if (classifier == 'lg'):
                cl = LogisticRegression()
            elif classifier=='dt':
                cl=tree.DecisionTreeClassifier()
            elif (classifier == 'rf'):
                cl = RandomForestClassifier()
            elif (classifier == 'svm'):
                cl = svm.SVC()
                #cl = svm.LinearSVC()
            elif (classifier == 'nnc'):
                cl = KNeighborsClassifier(n_neighbors=3)
            elif (classifier == 'nb'):
                cl = GaussianNB()
            elif (classifier == 'nnet'):
                cl = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
            else:
                print('error')
                exit()
            
            model=cl.fit(X,Y)
            predictions = model.predict(X_test)
            
            df_temp = X_test.copy()
            df_temp['predictions'] = predictions  
            df_temp[label] = Y_test
            
            if (classifier == 'svm'):
                # since auc is not possible with svm, put anything:
                fpr_roc_1 = [0.0,0.0]
                tpr_roc_1 = [0.0,0.0]
                auc_1 = 0.0
                fpr_roc_0 = [0.0,0.0]
                tpr_roc_0 = [0.0,0.0]
                auc_0 = 0.0
            else:
                y_pred_proba = model.predict_proba(X_test)[::,1]
                df_temp['pred_proba'] = y_pred_proba
                
                fpr_roc, tpr_roc, _ = metrics.roc_curve(Y_test,  y_pred_proba)

                #df_temp_X_1 = df_temp.loc[(df_temp[sensitive]==1)].drop(label, axis=1)
                df_temp_Y_1 = df_temp.loc[(df_temp[sensitive]==1)][label]
                df_temp_pred_proba_1 = df_temp.loc[(df_temp[sensitive]==1)]['pred_proba']
                fpr_roc_1, tpr_roc_1, _ = metrics.roc_curve(df_temp_Y_1,  df_temp_pred_proba_1)
                auc_1 = metrics.roc_auc_score(df_temp_Y_1,  df_temp_pred_proba_1)

                df_temp_Y_0 = df_temp.loc[(df_temp[sensitive]==0)][label]
                df_temp_pred_proba_0 = df_temp.loc[(df_temp[sensitive]==0)]['pred_proba']
                fpr_roc_0, tpr_roc_0, _ = metrics.roc_curve(df_temp_Y_0,  df_temp_pred_proba_0)
                auc_0 = metrics.roc_auc_score(df_temp_Y_0,  df_temp_pred_proba_0)
#                
            fpr0 = df_temp.loc[(df_temp[sensitive]==0)&(df_temp.predictions ==1)&(df_temp[label] == 0)].shape[0]/\
                    df_temp.loc[(df_temp[sensitive]==0)&(df_temp[label] == 0)].shape[0]  
            fpr1 = df_temp.loc[(df_temp[sensitive]==1)&(df_temp.predictions ==1)&(df_temp[label] == 0)].shape[0]/\
                    df_temp.loc[(df_temp[sensitive]==1)&(df_temp[label] == 0)].shape[0]
#             fpr = df_temp.loc[(df_temp.predictions ==1)&(df_temp[label] == 0)].shape[0]/\
#                     df_temp.loc[(df_temp[label] == 0)].shape[0]
            
            fnr0 = df_temp.loc[(df_temp[sensitive]==0)&(df_temp.predictions ==0)&(df_temp[label] == 1)].shape[0]/\
                    df_temp.loc[(df_temp[sensitive]==0)&(df_temp[label] == 1)].shape[0]  
            fnr1 = df_temp.loc[(df_temp[sensitive]==1)&(df_temp.predictions ==0)&(df_temp[label] == 1)].shape[0]/\
                    df_temp.loc[(df_temp[sensitive]==1)&(df_temp[label] == 1)].shape[0]
            
            eo0 = df_temp.loc[(df_temp[sensitive]==0)&(df_temp.predictions ==1)&(df_temp[label] == 1)].shape[0]/\
                    df_temp.loc[(df_temp[sensitive]==0)&(df_temp[label] == 1)].shape[0]  
            eo1 = df_temp.loc[(df_temp[sensitive]==1)&(df_temp.predictions ==1)&(df_temp[label] == 1)].shape[0]/\
                    df_temp.loc[(df_temp[sensitive]==1)&(df_temp[label] == 1)].shape[0]            
            
            sd0 = df_temp.loc[(df_temp[sensitive]==0)&(df_temp.predictions ==1)].shape[0]/\
            df_temp.loc[df_temp[sensitive]==0].shape[0]
            sd1 = df_temp.loc[(df_temp[sensitive]==1)&(df_temp.predictions ==1)].shape[0]/\
            df_temp.loc[df_temp[sensitive]==1].shape[0]
            
            g0 = df_temp.loc[(df_temp[sensitive] == 0)]
            zol0 = (g0['predictions'] ^ g0[label]).sum()/g0.shape[0]
            
            g1 = df_temp.loc[(df_temp[sensitive] == 1)]
            zol1 = (g1['predictions'] ^ g1[label]).sum()/g1.shape[0]
           
            fpr0_temp.append(fpr0);
            fpr1_temp.append(fpr1);
           
            fnr0_temp.append(fnr0);
            fnr1_temp.append(fnr1);
            
            eo0_temp.append(eo0)
            eo1_temp.append(eo1)
            
            sd0_temp.append(sd0)
            sd1_temp.append(sd1)
            
            zol0_temp.append(zol0)
            zol1_temp.append(zol1)
            
            fpr0_roc_temp.append(fpr_roc_0)
            fpr1_roc_temp.append(fpr_roc_1)
            tpr0_roc_temp.append(tpr_roc_0)
            tpr1_roc_temp.append(tpr_roc_1)
            
            auc0_temp.append(auc_0)
            auc1_temp.append(auc_1)
            
            Disc_FPR_temp.append(fpr1 - fpr0)
            Disc_ZOL_temp.append(zol1 - zol0)
            Disc_EO_temp.append(eo1 - eo0)
            Disc_AUC_temp.append(auc_1 - auc_0)
            Disc_SD_temp.append(sd1 - sd0)
            
            # Leaving the following block here means that feature importance is computed
            # for every sample. This will take a lot of time. For samples larger than 200
            # This will take inacceptable amount of time. Instead, it should be commented 
            # and the feature importance is computed only once (further below)
            
            if (sensitiveImportance):# and (classifier != 'svm')):
                explainer = shap.explainers.Exact(model.predict, X)
                shap_values = explainer(X)
                shapVal_temp.append(get_importance(sensitive,shap_values))
            else:
                shapVal_temp.append(0.0)
                
         
       
        FPR0[key] = statistics.mean(fpr0_temp)
        FPR1[key] = statistics.mean(fpr1_temp)
        
        FNR0[key] = statistics.mean(fnr0_temp)
        FNR1[key] = statistics.mean(fnr1_temp)

        EO0[key] = statistics.mean(eo0_temp)
        EO1[key] = statistics.mean(eo1_temp)

        SD0[key] = statistics.mean(sd0_temp)
        SD1[key] = statistics.mean(sd1_temp)
        
        ZOL0[key] = statistics.mean(zol0_temp)
        ZOL1[key] = statistics.mean(zol1_temp)
        
        VarFPR0[key] = statistics.variance(fpr0_temp)
        VarFPR1[key] = statistics.variance(fpr1_temp)
        
        VarFNR0[key] = statistics.variance(fnr0_temp)
        VarFNR1[key] = statistics.variance(fnr1_temp)
        
        VarEO0[key] = statistics.variance(eo0_temp)
        VarEO1[key] = statistics.variance(eo1_temp)
        
        VarSD0[key] = statistics.variance(sd0_temp)
        VarSD1[key] = statistics.variance(sd1_temp)
        
        VarZOL0[key] = statistics.variance(zol0_temp)
        VarZOL1[key] = statistics.variance(zol1_temp)
        
        fpr_roc_data0[key] = roc_mean(fpr0_roc_temp)
        fpr_roc_data1[key] = roc_mean(fpr1_roc_temp)
        
        tpr_roc_data0[key] = roc_mean(tpr0_roc_temp)
        tpr_roc_data1[key] = roc_mean(tpr1_roc_temp)
        
        auc0[key] = statistics.mean(auc0_temp)
        auc1[key] = statistics.mean(auc1_temp)
        
        Var_auc0[key] = statistics.variance(auc0_temp)
        Var_auc1[key] = statistics.variance(auc1_temp)
        
        shapVal[key] = statistics.mean(shapVal_temp)
        varShapVal[key] = statistics.variance(shapVal_temp)
        
        # If feature importance is computed only for one sample,
        # comment the above block about feature importance and
        # uncomment the below block.
        
#         if (sensitiveImportance): # and (classifier != 'svm')):
#             explainer = shap.explainers.Exact(model.predict, X)
#             shap_values = explainer(X)
#             shapVal[key] = get_importance(sensitive,shap_values)
#         else:
#             shapVal[key] = 0.0
        
#         varShapVal[key] = 0.0
        
        VarDisc_FPR[key] = statistics.variance(Disc_FPR_temp)
        VarDisc_ZOL[key] = statistics.variance(Disc_ZOL_temp)
        VarDisc_EO[key] = statistics.variance(Disc_EO_temp)
        VarDisc_AUC[key] = statistics.variance(Disc_AUC_temp)
        VarDisc_SD[key] = statistics.variance(Disc_SD_temp)
    
       
    
    return FPR0, FPR1, FNR0, FNR1, EO0, EO1, SD0, SD1, ZOL0, ZOL1, VarFPR0, VarFPR1,\
            VarFNR0, VarFNR1, VarEO0, VarEO1, VarSD0, VarSD1, VarZOL0, VarZOL1, \
            fpr_roc_data0, fpr_roc_data1, tpr_roc_data0, tpr_roc_data1, auc0, auc1, \
            Var_auc0, Var_auc1, shapVal, varShapVal, \
            VarDisc_FPR, VarDisc_EO, VarDisc_ZOL, VarDisc_AUC, VarDisc_SD
  

In [47]:
def roc_mean(roc_data):
    #Get the smallest list
    minLength = math.inf
    for i in range(len(roc_data)):
        if (len(roc_data[i]) < minLength):
            minLength = len(roc_data[i])
    
    avg_roc = []
    for i in range(minLength):
        tot = 0
        for j in range(len(roc_data)):
            tot += roc_data[j][i]
        avg_roc.append(tot / len(roc_data))
    
    return avg_roc
        
   

In [35]:
# This cell is for testing purposes for the classifier_biases function.

dataset = 'Adult'

if (dataset == 'synthetic-complex'):
    df_pop_origin = experiment_data_complex(0.5,30000)
    dataset = 'synthetic-complex'
    df_pop = df_pop_origin
    sensitive = 'A'
    label = 'Y'
elif (dataset == 'synthetic-simple'):
    df_pop_origin = experiment_data_simple(0.5,30000)
    dataset = 'synthetic-simple'
    df_pop = df_pop_origin
    sensitive = 'A'
    label = 'Y'
elif (dataset == 'synthetic-new'):
    p=0.5 #proportion of each sensitive group
    n=30000 #data size
    w1,w2 = (0.6,0.2) #equal contribution (have to add up to 0.8)
    df_pop_origin = class_data(p,w1,w2, n)
    df_pop = df_pop_origin
    sensitive = 'A'
    label = 'Y'
elif (dataset == 'Adult'):
    df_pop_origin = pd.read_csv('Adult/adult_cleaned_bin.csv')
    df_pop = df_pop_origin
    sensitive = 'Sex'
    label = 'income'
elif (dataset == 'Compas'):
    df_pop_origin = pd.read_csv('Compas/compas_simplified.csv')
    df_pop = df_pop_origin
    dataset = 'Compas'
    sensitive = 'race'
    label = 'two_year_recid'
elif (dataset == 'Dutch'):
    df_pop_origin = pd.read_csv('Dutch/dutch_simplified.csv')
    df_pop = df_pop_origin
    dataset = 'Dutch'
    sensitive = 'sex'
    label = 'occupation'
elif (dataset == 'German'):
    df_pop_origin = pd.read_csv('German/german_simplified.csv')
    df_pop = df_pop_origin
    dataset = 'German'
    sensitive = 'personal_status_sex'
    label = 'default'
elif (dataset == 'Default'):
    df_pop_origin = pd.read_csv('Default/UCI_Credit_Card_Processed.csv')
    df_pop = df_pop_origin.replace({'SEX': {1:0, 2:1}})
    dataset = 'Default'
    sensitive = 'SEX'
    label = 'DEF_PAY_NMO'
    

y = df_pop[label]
X = df_pop.drop(label, axis = 1)

pop_train, pop_test, y_poptrain, y_poptest = train_test_split(X, y, test_size=0.33, random_state=42)
pop_train[label]=y_poptrain

nbrSamples = 100
# seeds = [x for x in range(nbrSamples)]
# sizes = [x for x in range(10, 50, 5)] + [x for x in range(50, 100, 10)] + \
#         [x for x in range(100, 1000, 100)] +\
#         [x for x in range(1000, 9000, 1000)]
seeds = [1,2]
sizes = [100,200]


# proportionsf = [x for x in np.arange(0.000001,0.02,0.002)] + \
#                 [round(x/10,3) for x in np.arange(0.2,10,0.5)] + \
#                 [x for x in np.arange(0.98,0.9999,0.002)]
# proportionsf = proportionsf[1:]

#classifiers = ['lg','rf','nnc','nb','svm'] 
classifier = 'lg'

samples = sampling_overlaping(seeds, sizes, pop_train)

FPR0, FPR1, FNR0, FNR1, EO0, EO1, SD0, SD1, ZOL0, ZOL1, VarFPR0, VarFPR1,\
            VarFNR0, VarFNR1, VarEO0, VarEO1, VarSD0, VarSD1, VarZOL0, VarZOL1, \
            fpr_roc_data0, fpr_roc_data1, tpr_roc_data0, tpr_roc_data1, auc0, auc1, \
            Var_auc0, Var_auc1, shapVal, varShapVal, \
            VarDisc_FPR, VarDisc_EO, VarDisc_ZOL, VarDisc_AUC, VarDisc_SD\
            = classifier_biases(samples, pop_test,y_poptest, classifier)




A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


IntProgress(value=0, max=2)

## Cross Validation Functions

In [67]:
# Sampling functions for the URB and data augmentation experiments

# Sampling function for URB experiments
def sample_ratios(size, df, n):
    all_samples={}
    proportionsf = [round(x/10,2) for x in np.arange(0.2,10,0.2)]
    females=df[df[sensitive]==0].copy()
    males=df[df[sensitive]==1].copy()
    for p in proportionsf:
        samples=[]
        for i in range(n):
            f = females.sample(frac = 1)#shuffle
            #f = f.sample(frac = 1)#shuffle
            fn= round(size*p)
            sf = f.iloc[0:fn]
            m = males.sample(frac = 1)#shuffle
            m = m.sample(frac = 1)#shuffle
            fm=size-fn
            sm = m.iloc[0:fm]
            s=pd.concat([sm, sf], ignore_index=True, axis=0)
            samples.append(s)
        all_samples[p]=samples
    
    return all_samples


    

def generateSamples(size, df, proportion, n):
    samples = []
    p = proportion
        
    for i in range(n):
            females=df[df[sensitive]==0].copy().sample(frac = 1)
            males=df[df[sensitive]==1].copy().sample(frac = 1)
            goodSample = False
            attempts = 0
            while ((goodSample == False) and (attempts < 100)):
                sf = females.sample(round(p*size), replace=True)
                sm = males.sample(round(size-(p*size)), replace=True)
                if (validSample(sf) and validSample(sm)):
                    goodSample = True
                    #print('attempt: ' + str(attempts))
                attempts += 1
            s=pd.concat([sm, sf], ignore_index=True, axis=0)
            #s = s.sample(frac = 1)
            samples.append(s)
    return samples

def sample_ratios_overlap(size, df, proportionsf, n):
    all_samples={}
    for p in proportionsf:
        all_samples[p] = generateSamples(size,df,p,n)
    
    return all_samples

# Fixing the privileged group size while changing the protected group size (randomly)
def sample_for_threshold(sizeG1, df, sizesG0, n):
    all_samples={}
    g1=df[df[sensitive]==1].copy().sample(frac = 1)
    g0=df[df[sensitive]==0].copy().sample(frac = 1)
    
    for sizeG0 in sizesG0:
        all_samples[sizeG0] = []
    
    for i in range(n):
        s1 = g1.sample(sizeG1, replace=True)
        for sizeG0 in sizesG0:
            s0 = g0.sample(sizeG0, replace=True)
            s = pd.concat([s1, s0], ignore_index=True, axis=0)
            s = s.sample(frac = 1)
            all_samples[sizeG0].append(s)
    
    return all_samples

# Fixing the privileged group size while changing the protected group size 
# by selecting only positive outcome samples from the protected group.
def sample_for_threshold_selective(sizeG1, df, sizesG0, n):
    all_samples={}
    g1=df[df[sensitive]==1].copy().sample(frac = 1)
    g0=df[df[sensitive]==0].copy().sample(frac = 1)
    g0p=df.loc[(df[sensitive]==0) & (df[label]==1)].copy().sample(frac = 1)
    #df_temp.loc[(df_temp[sensitive]==0)&(df_temp[label] == 1)]
    
    firstSize = sizesG0[0]
    
    for sizeG0 in sizesG0:
        all_samples[sizeG0] = []
    
    for i in range(n):
        s1 = g1.sample(sizeG1, replace=True)
        for sizeG0 in sizesG0:
            s0 = g0.sample(firstSize, replace=True)
            s0p = g0p.sample(sizeG0 - firstSize, replace = True)
            s = pd.concat([s1, s0, s0p], ignore_index=True, axis=0)
            s = s.sample(frac = 1)
            all_samples[sizeG0].append(s)
    
    return all_samples

# A different way to select samples.
def sample_for_threshold_v2(sizeG1, df, sizesG0, n):
    all_samples={}
    
#     females=df[df[sensitive]==0].copy()
#     males=df[df[sensitive]==1].copy()
    for sizeG0 in sizesG0:
        samples = []
        for i in range(n):
            g1=df[df[sensitive]==1].copy().sample(frac = 1)
            g0=df[df[sensitive]==0].copy().sample(frac = 1)
            s1 = g1.sample(sizeG1, replace=True)
            s0 = g0.sample(sizeG0, replace=True)
            s = pd.concat([s1, s0], ignore_index=True, axis=0)
            s = s.sample(frac = 1)
            samples.append(s)            
        
        all_samples[sizeG0] = samples
    
    return all_samples

# Fixing the protected group size while changing the privileged group size
def sample_for_threshold_inv(sizesG1, df, sizeG0, n):
    all_samples={}
    g1=df[df[sensitive]==1].copy().sample(frac = 1)
    g0=df[df[sensitive]==0].copy().sample(frac = 1)
    
    for sizeG1 in sizesG1:
        all_samples[sizeG1] = []
    
    for i in range(n):
        s0 = g0.sample(sizeG0, replace=True)
        for sizeG1 in sizesG1:
            s1 = g1.sample(sizeG1, replace=True)
            s = pd.concat([s1, s0], ignore_index=True, axis=0)
            s = s.sample(frac = 1)
            all_samples[sizeG1].append(s)
    
    return all_samples
    
    

In [ ]:
# Utility functions used in genDataCV function
def computeAverageFolds(key, m_list):

    nbrFolds = len(m_list)
    tot = 0
    for k in range(nbrFolds):
        tot += m_list[k][key]
    return (tot/nbrFolds)

def computeAverageRocs(key, roc_list):
    nbrFolds = len(roc_list)
    minLength = math.inf
    for j in range(nbrFolds):
        if (len(roc_list[j][key]) < minLength):
            minLength = len(roc_list[j][key])
    
    roc_list_avg = []

    for j in range(minLength):
        tot = 0
        for i in range(nbrFolds):
            tot += roc_list[i][key][j]
        roc_list_avg.append(tot/nbrFolds)
    
    return roc_list_avg
 

In [65]:
# main function for cross-validation. It generates the samples and calls classifier_biases function
def genDataCV(folds, m, sizes, seeds, classifier, task, sizeG = 0, sensitiveImportance=False):
    
    # task:
    # sizes
    # ratios
    
    
    df_pop_shuffule = df_pop.sample(frac=1)
    y = df_pop_shuffule[label]
    X = df_pop_shuffule.drop(label, axis = 1)
    kf = KFold(n_splits=folds)
    nbrFolds = kf.get_n_splits(X)
    
    nbrSamples = len(seeds)
    
    fpr0_folds = {}
    fpr1_folds = {}
    fnr0_folds = {}
    fnr1_folds = {}
    eo0_folds = {}
    eo1_folds = {}
    sd0_folds = {}
    sd1_folds = {}
    zol0_folds = {}
    zol1_folds = {}
    var_fpr0_folds = {}
    var_fpr1_folds = {}
    var_fnr0_folds = {}
    var_fnr1_folds = {}
    var_eo0_folds = {}
    var_eo1_folds = {}
    var_sd0_folds = {}
    var_sd1_folds = {}
    var_zol0_folds = {}
    var_zol1_folds = {}
    
    fpr_roc0_folds = {}
    fpr_roc1_folds = {}
    tpr_roc0_folds = {}
    tpr_roc1_folds = {}
    auc0_folds = {}
    auc1_folds = {}
    shapVal_folds = {}
    var_auc0_folds = {}
    var_auc1_folds = {}
    var_shapVal_folds = {}
    
    var_disc_fpr_folds = {}
    var_disc_eo_folds = {}
    var_disc_zol_folds = {}
    var_disc_auc_folds = {}
    var_disc_sd_folds = {}
    
    fpr0_list = []
    fpr1_list = []
    fnr0_list = []
    fnr1_list = []
    eo0_list = []
    eo1_list = []
    sd0_list = []
    sd1_list = []
    zol0_list = []
    zol1_list = []
    var_fpr0_list = []
    var_fpr1_list = []
    var_fnr0_list = []
    var_fnr1_list = []
    var_eo0_list = []
    var_eo1_list = []
    var_sd0_list = []
    var_sd1_list = []
    var_zol0_list = []
    var_zol1_list = []
    
    var_disc_fpr_list = []
    var_disc_eo_list = []
    var_disc_zol_list = []
    var_disc_auc_list = []
    var_disc_sd_list = []
    
    fpr_roc0_list = []
    fpr_roc1_list = []
    tpr_roc0_list = []
    tpr_roc1_list = []
    auc0_list = []
    auc1_list = []
    shapVal_list = []
    var_auc0_list = []
    var_auc1_list = []
    var_shapVal_list = []
    
    
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        pop_train = df_pop_shuffule.iloc[train_index]
        pop_test = df_pop_shuffule.iloc[test_index]
        y_test = pop_test[label]
        X_test = pop_test.drop(label, axis=1)
        #print(X_test)
        
        if (task == 'sizes'):
            samples = sampling_overlaping(seeds, sizes, pop_train)
        elif (task == 'ratios'):
            samples = sample_ratios_overlap(m, pop_train, sizes, nbrSamples)
        elif (task == 'threshold'):
            # Here, you select which sampling approach is used
            # (depends on which experiment is being performed).
            samples = sample_for_threshold(sizeG,pop_train,sizes, nbrSamples)
            #samples = sample_for_threshold_selective(sizeG,pop_train,sizes, nbrSamples)
            #samples = sample_for_threshold_inv(sizes,pop_train,sizeG, nbrSamples)
            
        
        FPR0, FPR1, FNR0, FNR1, EO0, EO1, SD0, SD1, ZOL0, ZOL1, VarFPR0, VarFPR1,\
            VarFNR0, VarFNR1, VarEO0, VarEO1, VarSD0, VarSD1, VarZOL0, VarZOL1, \
            fpr_roc0, fpr_roc1, tpr_roc0, tpr_roc1, auc0, auc1, \
            Var_auc0, Var_auc1, shapVal, varShapVal, \
            VarDisc_FPR, VarDisc_EO, VarDisc_ZOL, VarDisc_AUC, VarDisc_SD = \
            classifier_biases(samples, X_test, y_test, classifier,\
                              sensitiveImportance=sensitiveImportance)
        

        fpr0_list.append(FPR0)
        fpr1_list.append(FPR1)
        fnr0_list.append(FNR0)
        fnr1_list.append(FNR1)
        eo0_list.append(EO0)
        eo1_list.append(EO1)
        sd0_list.append(SD0)
        sd1_list.append(SD1)
        zol0_list.append(ZOL0)
        zol1_list.append(ZOL1)
        var_fpr0_list.append(VarFPR0)
        var_fpr1_list.append(VarFPR1)
        var_fnr0_list.append(VarFNR0)
        var_fnr1_list.append(VarFNR1)
        var_eo0_list.append(VarEO0)
        var_eo1_list.append(VarEO1)
        var_sd0_list.append(VarSD0)
        var_sd1_list.append(VarSD1)
        var_zol0_list.append(VarZOL0)
        var_zol1_list.append(VarZOL1)
        fpr_roc0_list.append(fpr_roc0)
        fpr_roc1_list.append(fpr_roc1)
        tpr_roc0_list.append(tpr_roc0)
        tpr_roc1_list.append(tpr_roc1)
        auc0_list.append(auc0)
        auc1_list.append(auc1)
        shapVal_list.append(shapVal)
        var_auc0_list.append(Var_auc0)
        var_auc1_list.append(Var_auc1)
        var_shapVal_list.append(varShapVal)
        var_disc_fpr_list.append(VarDisc_FPR)
        var_disc_eo_list.append(VarDisc_EO)
        var_disc_zol_list.append(VarDisc_ZOL)
        var_disc_auc_list.append(VarDisc_AUC)
        var_disc_sd_list.append(VarDisc_SD)
  
    for key in list(FPR0.keys()):
        
        fpr0_folds[key] = computeAverageFolds(key, fpr0_list) 
        fpr1_folds[key] = computeAverageFolds(key, fpr1_list)
        fnr0_folds[key] = computeAverageFolds(key, fnr0_list)
        fnr1_folds[key] = computeAverageFolds(key, fnr1_list)
        eo0_folds[key] = computeAverageFolds(key, eo0_list)
        eo1_folds[key] = computeAverageFolds(key, eo1_list)
        sd0_folds[key] = computeAverageFolds(key, sd0_list)
        sd1_folds[key] = computeAverageFolds(key, sd1_list)
        zol0_folds[key] = computeAverageFolds(key, zol0_list)
        zol1_folds[key] = computeAverageFolds(key, zol1_list)
        

        var_fpr0_folds[key] = computeAverageFolds(key, var_fpr0_list)
        var_fpr1_folds[key] = computeAverageFolds(key, var_fpr1_list)
        var_fnr0_folds[key] = computeAverageFolds(key, var_fnr0_list)
        var_fnr1_folds[key] = computeAverageFolds(key, var_fnr1_list)
        var_eo0_folds[key] = computeAverageFolds(key, var_eo0_list)
        var_eo1_folds[key] = computeAverageFolds(key, var_eo1_list)
        var_sd0_folds[key] = computeAverageFolds(key, var_sd0_list)
        var_sd1_folds[key] = computeAverageFolds(key, var_sd1_list)
        var_zol0_folds[key] = computeAverageFolds(key, var_zol0_list)
        var_zol1_folds[key] = computeAverageFolds(key, var_zol1_list)
                
        fpr_roc0_folds[key] = computeAverageRocs(key, fpr_roc0_list)
        fpr_roc1_folds[key] = computeAverageRocs(key, fpr_roc1_list)
        tpr_roc0_folds[key] = computeAverageRocs(key, tpr_roc0_list)
        tpr_roc1_folds[key] = computeAverageRocs(key, tpr_roc1_list)
        auc0_folds[key] = computeAverageFolds(key, auc0_list)
        auc1_folds[key] = computeAverageFolds(key, auc1_list)
        shapVal_folds[key] = computeAverageFolds(key, shapVal_list)
        var_auc0_folds[key] = computeAverageFolds(key, var_auc0_list)
        var_auc1_folds[key] = computeAverageFolds(key, var_auc1_list)
        var_shapVal_folds[key] = computeAverageFolds(key, var_shapVal_list)
        
        var_disc_fpr_folds[key] = computeAverageFolds(key, var_disc_fpr_list)
        var_disc_eo_folds[key] = computeAverageFolds(key, var_disc_eo_list)
        var_disc_zol_folds[key] = computeAverageFolds(key, var_disc_zol_list)
        var_disc_auc_folds[key] = computeAverageFolds(key, var_disc_auc_list)
        var_disc_sd_folds[key] = computeAverageFolds(key, var_disc_sd_list)
        
    return fpr0_folds, fpr1_folds, fnr0_folds, fnr1_folds, eo0_folds, eo1_folds, sd0_folds, sd1_folds,\
            zol0_folds, zol1_folds, var_fpr0_folds, var_fpr1_folds, var_fnr0_folds, var_fnr1_folds, \
            var_eo0_folds, var_eo1_folds, var_sd0_folds, var_sd1_folds, var_zol0_folds, var_zol1_folds,\
            fpr_roc0_folds, fpr_roc1_folds, tpr_roc0_folds, tpr_roc1_folds, auc0_folds, auc1_folds, \
            var_auc0_folds, var_auc1_folds, shapVal_folds, var_shapVal_folds, \
            var_disc_fpr_folds, var_disc_eo_folds, var_disc_zol_folds, var_disc_auc_folds, var_disc_sd_folds
   

In [44]:
def compute_SSB_URB(meas0, meas1, ref0, ref1):

    experiments = list(meas0.keys())

    SSURB = {}
    Disc = {}
    disc_ref = ref1 - ref0
    
    for k in experiments:
        Disc[k] = meas1[k] - meas0[k]
        SSURB[k] = meas1[k] - meas0[k] - disc_ref       

    return Disc, disc_ref, SSURB


# Main script for all experiments in the paper
This script carries out experiment and save the obtained data structures (mainly dictionary) in files.
These files can be retrieved for plotting purposes.

In [ ]:
# Set the parameters for the desired experiment:

# Select which experiment to perform (task):
# sizes: SSB experiment
# ratios: URB experiment
# threshold: Data augmentation experiment

task = 'threshold'

#dataset = 'Dutch'
#datasets = ['synthetic-complex', 'synthetic-simple', 'Adult-bin','Adult-not-bin','Compas','Dutch']#,'German']
#datasets = ['Adult-bin','Compas','Dutch', 'Adult-not-bin','synthetic-complex', 'synthetic-simple', 'synthetic-new']#,'German']
datasets = ['Dutch', 'Adult-bin']

# Whether to save the results in files
fileSaving = True

# Size of the training set for the URB experiment
m = 1000

# How many times each instance of an experiment is repeated
# (Finally the average of all those experiments is kept)
# (This will also allow to compute variance)
nbrSamples = 50
seeds = [x for x in range(nbrSamples)]

# Sample sizes for the SSB experiment
#     sizes = [x for x in range(10, 50, 5)] + [x for x in range(50, 100, 10)] + \
#             [x for x in range(100, 1000, 100)] +\
#             [x for x in range(1000, 9000, 1000)]
sizes = [15, 20, 25, 30, 35, 40, 45, 60, 80, 100, \
         120, 140, 160, 210, 260, 310, 360, 410, \
         460, 540, 640, 740, 840, 940, 1080, 1280, \
         1480, 1680, 1880, 2150, 2650, 3150, 3650, \
         4150, 4650, 5400, 6400, 7400, 8400]


# Sizes for the data augmentation experiment
sizesThreshold = [x for x in range(1, 100, 2)]

# Size of the portion (typically privileged group) to stay fixed in the data augmentation experiment
singleSize = 100

# Proportions for the URB experiment
#proportionsf = [round(x/10,2) for x in np.arange(0.2,10,0.2)]
proportionsf = [x for x in np.arange(0.000001,0.02,0.002)] + \
                [round(x/10,3) for x in np.arange(0.2,10,0.5)] + \
                [x for x in np.arange(0.98,0.9999,0.002)]
proportionsf = proportionsf[1:]


# Select the classifiers to be used
#classifiers = ['lg','dt','rf','nnc','nb','svm'] 
classifiers = ['lg','dt']#,'rf','nnc', 'svm'] 
#classifiers = ['lg']

# Number of folds for the cross-validation
nbrFolds = 3

# Estimate sensitive feature importance using SHAP
# Warning: this can slow experiments significantly
computeImportance = False


for dataset in datasets:
    print(dataset)

    if (dataset == 'synthetic-complex'):
        df_pop_origin = experiment_data_complex(0.5,30000)
        dataset = 'synthetic-complex'
        df_pop = df_pop_origin
        sensitive = 'A'
        label = 'Y'
    elif (dataset == 'synthetic-simple'):
        df_pop_origin = experiment_data_simple(0.5,30000)
        dataset = 'synthetic-simple'
        df_pop = df_pop_origin
        sensitive = 'A'
        label = 'Y'
    elif (dataset == 'synthetic-new'):
        p=0.5 #proportion of each sensitive group
        n=30000 #data size
        w1,w2 = (0.6,0.2) #equal contribution (have to add up to 0.8)
        df_pop_origin = class_data(p,w1,w2, n)
        df_pop = df_pop_origin
        sensitive = 'A'
        label = 'Y'
    elif (dataset == 'Adult-bin'):
        df_pop_origin = pd.read_csv('Adult/adult_cleaned_bin.csv')
        df_pop = df_pop_origin
        sensitive = 'Sex'
        label = 'income'
    elif (dataset == 'Adult-not-bin'):
        df_pop_origin = pd.read_csv('Adult/adult_cleaned_not_bin.csv')
        df_pop = df_pop_origin
        sensitive = 'Sex'
        label = 'income'
    elif (dataset == 'Compas'):
        df_pop_origin = pd.read_csv('Compas/compas_simplified.csv')
        df_pop = df_pop_origin
        dataset = 'Compas'
        sensitive = 'race'
        label = 'two_year_recid'
    elif (dataset == 'Dutch'):
        df_pop_origin = pd.read_csv('Dutch/dutch_simplified.csv')
        df_pop = df_pop_origin
        dataset = 'Dutch'
        sensitive = 'sex'
        label = 'occupation'
    elif (dataset == 'German'):
        df_pop_origin = pd.read_csv('German/german_simplified.csv')
        df_pop = df_pop_origin
        dataset = 'German'
        sensitive = 'personal_status_sex'
        label = 'default'


    y = df_pop[label]
    X = df_pop.drop(label, axis = 1)
    pop_train, pop_test, y_poptrain, y_poptest = train_test_split(X, y, test_size=0.33, random_state=42)
    pop_train[label]=y_poptrain



    if(task == 'sizes'):
        experiments = sizes
    if(task == 'ratios'):
        experiments = proportionsf
    if(task == 'threshold'):
        experiments = sizesThreshold

    for i in range(len(classifiers)):   
   
        classifier = classifiers[i]
    
        if (task == 'sizes'):
            print( 'nbr Samples = ' + str(nbrSamples) + ', classifier: ' + classifier +
              ', nbr Folds = ' + str(nbrFolds))
        elif (task == 'ratios'):    
            print( 'nbr Samples = ' + str(nbrSamples) + ', classifier: ' + classifier + \
                  ', sample size = ' + str(m) + ', nbr Folds = ' + str(nbrFolds))
        elif (task == 'threshold'):    
            print( 'nbr Samples = ' + str(nbrSamples) + ', classifier: ' + classifier + \
                  ', G1 size = ' + str(singleSize) + ', nbr Folds = ' + str(nbrFolds))

        fpr0, fpr1, fnr0, fnr1, eo0, eo1, sd0, sd1,\
                zol0, zol1, var_fpr0, var_fpr1, var_fnr0, var_fnr1, \
                var_eo0, var_eo1, var_sd0, var_sd1, var_zol0, var_zol1,\
                fpr_roc0, fpr_roc1, tpr_roc0, tpr_roc1, auc0, auc1, \
                var_auc0, var_auc1, shapVal, var_shapVal,\
                var_disc_fpr, var_disc_eo, var_disc_zol, var_disc_auc, var_disc_sd= \
            genDataCV(nbrFolds, m, experiments, seeds, classifier, task, \
                      sizeG = singleSize, sensitiveImportance=computeImportance)

        if (task == 'sizes' or task == 'threshold'):
            ref = math.floor(df_pop.shape[0] * ((nbrFolds-1)/nbrFolds))
        if (task == 'ratios'):
            fr,mr = getPopRatio() 
            ref = fr


        experimentRef = [ref]
        if (task == 'threshold'):
            taskTh = 'sizes'
        else:
            taskTh = task

        # Compute the same quantities for the reference scenario
        # Reference is used mainly for SSB and URB experiments
        fpr0_ref_d, fpr1_ref_d, fnr0_ref_d, fnr1_ref_d, eo0_ref_d, eo1_ref_d, sd0_ref_d, sd1_ref_d,\
        zol0_ref_d, zol1_ref_d, var_fpr0_ref_d, var_fpr1_ref_d, var_fnr0_ref_d, var_fnr1_ref_d, \
        var_eo0_ref_d, var_eo1_ref_d, var_sd0_ref_d, var_sd1_ref_d, var_zol0_ref_d, var_zol1_ref_d, \
        fpr_roc0_ref_d, fpr_roc1_ref_d, tpr_roc0_ref_d, tpr_roc1_ref_d, auc0_ref_d, auc1_ref_d, \
        var_auc0_ref_d, var_auc1_ref_d, shapVal_ref_d, var_shapVal_ref_d,\
        var_disc_fpr_ref_d, var_disc_eo_ref_d, var_disc_zol_ref_d, var_disc_auc_ref_d, var_disc_sd_ref_d = \
                    genDataCV(nbrFolds, m, experimentRef, seeds, classifier, taskTh, sizeG = singleSize)

        fpr0_ref = fpr0_ref_d[ref]; fpr1_ref = fpr1_ref_d[ref]; fnr0_ref = fnr0_ref_d[ref]; 
        fnr1_ref = fnr1_ref_d[ref]; eo0_ref = eo0_ref_d[ref]; eo1_ref = eo1_ref_d[ref];
        sd0_ref = sd0_ref_d[ref]; sd1_ref = sd1_ref_d[ref]; zol0_ref = zol0_ref_d[ref]; zol1_ref = zol1_ref_d[ref]
        var_fpr0_ref = var_fpr0_ref_d[ref];var_fpr1_ref = var_fpr1_ref_d[ref];
        var_fnr0_ref = var_fnr0_ref_d[ref];var_fnr1_ref = var_fnr1_ref_d[ref];
        var_eo0_ref = var_eo0_ref_d[ref];var_eo1_ref = var_eo1_ref_d[ref];
        var_sd0_ref = var_sd0_ref_d[ref];var_sd1_ref = var_sd1_ref_d[ref];
        var_zol0_ref = var_zol0_ref_d[ref];var_zol1_ref = var_zol1_ref_d[ref];
        
        fpr_roc0_ref = fpr_roc0_ref_d[ref]; fpr_roc1_ref = fpr_roc1_ref_d[ref];
        tpr_roc0_ref = tpr_roc0_ref_d[ref]; tpr_roc1_ref = tpr_roc1_ref_d[ref];
        auc0_ref = auc0_ref_d[ref]; auc1_ref = auc1_ref_d[ref]; var_auc0_ref = var_auc0_ref_d[ref];\
        var_auc1_ref = var_auc1_ref_d[ref]; shapVal_ref = shapVal_ref_d[ref]; var_shapVal_ref = var_shapVal_ref_d[ref]
        var_disc_fpr_ref = var_disc_fpr_ref_d[ref]; var_disc_eo_ref = var_disc_eo_ref_d[ref];
        var_disc_zol_ref = var_disc_zol_ref_d[ref]; var_disc_auc_ref = var_disc_auc_ref_d[ref];
        var_disc_sd_ref = var_disc_sd_ref_d[ref]

        D_fpr, D_fpr_ref, SSURB_fpr = compute_SSB_URB(fpr0, fpr1, fpr0_ref, fpr1_ref)
        D_fnr, D_fnr_ref, SSURB_fnr = compute_SSB_URB(fnr0, fnr1, fnr0_ref, fnr1_ref)
        D_eo, D_eo_ref, SSURB_eo = compute_SSB_URB(eo0, eo1, eo0_ref, eo1_ref)
        D_sd, D_sd_ref, SSURB_sd = compute_SSB_URB(sd0, sd1, sd0_ref, sd1_ref)
        D_zol, D_zol_ref, SSURB_zol = compute_SSB_URB(zol0, zol1, zol0_ref, zol1_ref)
        D_var_fpr, D_var_fpr_ref, SSURB_var_fpr = compute_SSB_URB(var_fpr0, var_fpr1, \
                                                                  var_fpr0_ref, var_fpr1_ref)
        D_var_fnr, D_var_fnr_ref, SSURB_var_fnr = compute_SSB_URB(var_fnr0, var_fnr1, \
                                                                  var_fnr0_ref, var_fnr1_ref)
        D_var_eo, D_var_eo_ref, SSURB_var_eo = compute_SSB_URB(var_eo0, var_eo1, \
                                                               var_eo0_ref, var_eo1_ref)
        D_var_sd, D_var_sd_ref, SSURB_var_sd = compute_SSB_URB(var_sd0, var_sd1, \
                                                               var_sd0_ref, var_sd1_ref)
        D_var_zol, D_var_zol_ref, SSURB_var_zol = compute_SSB_URB(var_zol0, var_zol1, \
                                                                  var_zol0_ref, var_zol1_ref)

   
        if (fileSaving):
            currentDirectory = '/Users/sami/GitHub/RepresentationBias/savedResults/'
            folderName = time.strftime('%Y%m%d-%H%M%S-' + dataset + '-' + classifier + '-' + task + '-sensContr-30/')
            os.mkdir(currentDirectory + folderName)
            if (task == 'ratios'):
                filename = 'ratiosData-'  + str(m) + '-' + classifier + '.pkl'
            if (task == 'sizes'):
                filename = 'sizesData-' + classifier + '.pkl'
            if (task == 'threshold'):
                filename = 'thresholdData-' + classifier + '.pkl'
            fullfilename = currentDirectory + folderName + filename 

            plottingData = (fpr0, fpr1, fnr0, fnr1, eo0, eo1, \
                            sd0, sd1, zol0, zol1,
                            var_fpr0, var_fpr1, var_fnr0, var_fnr1, var_eo0,\
                            var_eo1, var_sd0, var_sd1, var_zol0, var_zol1, \
                            fpr_roc0, fpr_roc1, tpr_roc0, tpr_roc1, auc0, auc1,\
                            var_auc0, var_auc1, shapVal, var_shapVal,\
                            var_disc_fpr, var_disc_eo, var_disc_zol, var_disc_auc, var_disc_sd,\
                            fpr0_ref, fpr1_ref, fnr0_ref, fnr1_ref, eo0_ref, eo1_ref, sd0_ref, sd1_ref,\
                            zol0_ref, zol1_ref, var_fpr0_ref, var_fpr1_ref, var_fnr0_ref, var_fnr1_ref, \
                            var_eo0_ref, var_eo1_ref, var_sd0_ref, var_sd1_ref, var_zol0_ref, var_zol1_ref, \
                            fpr_roc0_ref, fpr_roc1_ref, tpr_roc0_ref, tpr_roc1_ref, auc0_ref, auc1_ref, \
                            var_auc0_ref, var_auc1_ref, shapVal_ref, var_shapVal_ref, \
                            var_disc_fpr_ref, var_disc_eo_ref, var_disc_zol_ref, var_disc_auc_ref, var_disc_sd_ref, \
                            task, nbrSamples, dataset, classifier)

            with open(fullfilename, 'wb') as f:
                pickle.dump(plottingData, f)
